In [1]:
from ctypes import *
import numpy as np
import gym
from gym import error
from gym.utils import closer
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common import make_vec_env
from stable_baselines import A2C, PPO2
from gym import spaces
from pypylon import pylon
import stable_baselines
import time
from PIL import Image

C:\Users\CIG\Anaconda3\lib\site-packages\dask\dataframe\utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
class BaslerEnv(gym.Env):
    """A goal-based environment. It functions just as any regular OpenAI Gym environment but it
    imposes a required structure on the observation_space. More concretely, the observation
    space is required to contain at least three elements, namely `observation`, `desired_goal`, and
    `achieved_goal`. Here, `desired_goal` specifies the goal that the agent should attempt to achieve.
    `achieved_goal` is the goal that it currently achieved instead. `observation` contains the
    actual observations of the environment as per usual.
    """
    # Set this in SOME subclasses
    metadata = {'render.modes': ['human']}

    
    def __init__(self, camera, GainRaw = 36, ExposureTimeRaw = 718977, AcquisitionFrameRateAbs = 30, threshold = -24):
        
        # Create an instant camera object with the camera device found first.
        #self.camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
        #self.camera.Open()
        
        self.camera = camera
        
        # https://docs.baslerweb.com/features
        self.camera.GainRaw.Value = GainRaw
        self.camera.ExposureTimeRaw.Value = ExposureTimeRaw
        self.camera.AcquisitionFrameRateAbs.Value = AcquisitionFrameRateAbs
        
        # corning lib
        self.lib = cdll.LoadLibrary(r"C:\Users\CIG\Documents\MATLAB\ComCasp64.dll")
        #Check if Maxim driver dll is loaded
        eCOMCaspErr = getattr(self.lib,'Casp_OpenCOM')
        print('eCOMCaspErr:', eCOMCaspErr(), self.lib.Casp_OpenCOM())
        
        # Focus Voltage
        self.action_space = spaces.Discrete(69)
        
         
        # get width and height from camera for making obs space
        width = self.camera.Width.GetValue()
        height = self.camera.Height.GetValue()
        print(width, height)
        
        # image
        self.observation_space = spaces.Box(low=0, high=255, 
                                             shape=(height, width, 3), 
                                             dtype=np.float16)
        
        # code from https://github.com/basler/pypylon/blob/master/samples/opencv.py
        img = pylon.PylonImage()
        self.converter = pylon.ImageFormatConverter()
        
        # converting to opencv bgr format
        self.converter.OutputPixelFormat = pylon.PixelType_BGR8packed
        self.converter.OutputBitAlignment = pylon.OutputBitAlignment_MsbAligned
        
        self.threshold = threshold
        self.i = 0
        
        time.sleep(1)

    def reset(self):
        x = c_double(24)
        self.lib.Casp_SetFocusVoltage(x)
        
        
    def compute_reward(self, image, size=60):
        """Compute the step reward. This externalizes the reward function and makes
        it dependent on a desired goal and the one that was achieved. If you wish to include
        additional rewards that are independent of the goal, you can include the necessary values
        to derive it in 'info' and compute it accordingly.
        Args:
            achieved_goal (object): the goal that was achieved during execution
            desired_goal (object): the desired goal that we asked the agent to attempt to achieve
            info (dict): an info dictionary with additional information
        Returns:
            float: The reward that corresponds to the provided achieved goal w.r.t. to the desired
            goal. Note that the following should always hold true:
                ob, reward, done, info = env.step()
                assert reward == env.compute_reward(ob['achieved_goal'], ob['goal'], info)
        """
        
        # code from https://www.pyimagesearch.com/2020/06/15/opencv-fast-fourier-transform-fft-for-blur-detection-in-images-and-video-streams/
        
        # grab the dimensions of the image and use the dimensions to
        # derive the center (x, y)-coordinates
        (h, w, m) = image.shape
        (cX, cY) = (int(w / 2.0), int(h / 2.0))

        fft = np.fft.fft2(image)
        fftShift = np.fft.fftshift(fft)

        # zero-out the center of the FFT shift (i.e., remove low
        # frequencies), apply the inverse shift such that the DC
        # component once again becomes the top-left, and then apply
        # the inverse FFT
        fftShift[cY - size:cY + size, cX - size:cX + size] = 0
        fftShift = np.fft.ifftshift(fftShift)
        recon = np.fft.ifft2(fftShift)

        # compute the magnitude spectrum of the reconstructed image,
        # then compute the mean of the magnitude values
        magnitude = 20 * np.log(np.abs(recon))
        mean = np.mean(magnitude)
        # the image will be considered "blurry" if the mean value of the
        # magnitudes is less than the threshold value
        return mean
    
    def grab_image(self):
        
        #https://github.com/basler/pypylon/blob/master/samples/opencv.py
        self.camera.StartGrabbing()
        while 1:
            grabResult = self.camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)

            if grabResult.GrabSucceeded():
                
                # Access the image data
                image = self.converter.Convert(grabResult)
                image = image.GetArray()
#                 print(img[0])
#                 print('shape:', img.shape)
#                 plt.imshow(img)
                break
    
        self.camera.StopGrabbing()
        return image
    
    
    def step(self, action):
        """Run one timestep of the environment's dynamics. When end of
        episode is reached, you are responsible for calling `reset()`
        to reset this environment's state.
        Accepts an action and returns a tuple (observation, reward, done, info).
        Args:
            action (object): an action provided by the agent
        Returns:
            observation (object): agent's observation of the current environment
            reward (float) : amount of reward returned after previous action
            done (bool): whether the episode has ended, in which case further step() calls will return undefined results
            info (dict): contains auxiliary diagnostic information (helpful for debugging, and sometimes learning)
        """
        
        # Focus Voltage Low value = 24
        if action < 25:
            action = 25
        
        x = c_double(action)
        self.lib.Casp_SetFocusVoltage(x)
        time.sleep(1)
        
        obs = self.grab_image()
        blurry = self.compute_reward(obs)

        done = bool(blurry < self.threshold)
        reward = -1
        
        im = Image.fromarray(obs)
        filename = "saved_pypylon_img_%d_%s.png" % (self.i, str(done))
        im.save(filename)
        self.i+=1
        
        return obs, reward, done, {}
        
        
    def close(self):
        """Override close in your subclass to perform any necessary cleanup.
        Environments will automatically close() themselves when
        garbage collected or when the program exits.
        """
        self.lib.Casp_CloseCOM()
        self.camera.Close()
        


In [3]:
camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
camera.Open()
env = BaslerEnv(camera)

eCOMCaspErr: 0 0
835 442


In [4]:
model = PPO2(MlpPolicy, env, tensorboard_log = '')





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





In [ ]:
model.learn(total_timesteps=300)

In [10]:
camera.Close()

In [2]:
#lib = cdll.LoadLibrary(r"C:\Users\CIG\Documents\MATLAB\ComCasp64.dll")

In [3]:
#lib

<CDLL 'C:\Users\CIG\Documents\MATLAB\ComCasp64.dll', handle 7ffd76590000 at 0x1bf9e31a700>

In [4]:
#eCOMCaspErr = getattr(lib,'Casp_OpenCOM')

In [5]:
#eCOMCaspErr()

0

In [6]:
#lib.Casp_OpenCOM()

0

In [8]:
#Once connection is etablish, Board settings may be changed.
#To change Focus base on Voltage value
#x = c_double(50)
#lib.Casp_SetFocusVoltage(x)

0

In [30]:
#eCOMCaspErr code can be converted to a string using Casp_GetErrorMsg() function

#lib.Casp_GetErrorMsg

0

In [9]:
#cleanup

#lib.Casp_CloseCOM()

0